In [2]:
from utils import *
import re
import random
from pprint import pprint
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
all_data = num_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

seed = int(random.uniform(0, 100))
train_ratio = 0.7

neg_train, neg_test, ytrain, ytest = train_test_split(neg_data, neg_data["labels"], train_size = train_ratio, stratify = neg_data["labels"], random_state = seed)
pos_train, pos_test, ytrain, ytest = train_test_split(pos_data, pos_data["labels"], train_size = train_ratio, stratify = pos_data["labels"], random_state = seed)
#neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
#pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

#neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
#pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [3]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words= stopwords_pt)

cv.fit(neg_train["texts"])
vocab_neg = set(cv.vocabulary_.keys())

cv.fit(pos_train["texts"])
vocab_pos = set(cv.vocabulary_.keys())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2857
Dimensão vocabulario positivo: 2376


In [4]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))
print("Dimensão: ", len(lexicon))

Dimensão:  3013


In [7]:
lexicon


['continuou',
 'lamentei',
 'desfile',
 'entregou',
 'defendida',
 'mão',
 'reuniu',
 'terror',
 'justamente',
 'jovens',
 'leilões',
 'sujo',
 'destinados',
 'concretas',
 'incentivado',
 'alardeada',
 'europeus',
 'admirá',
 'ruim',
 'difundir',
 'esperada',
 'funcione',
 'pe',
 'pressionada',
 'autônomo',
 'hesitante',
 'católica',
 'precisa',
 'encostada',
 'dispondo',
 'relatadas',
 'ridícula',
 'negociação',
 'sites',
 'senadores',
 'convictos',
 'manifestantes',
 'direcionou',
 'tido',
 'populações',
 'custou',
 'jurídico',
 'recessivo',
 'obrigada',
 'rebateu',
 'comportando',
 'parecem',
 'experimentada',
 'critica',
 'expulsa',
 'acha',
 'simplificação',
 'aumentos',
 'rituais',
 'esquecimento',
 'eólica',
 'familiar',
 'havido',
 'telhado',
 'enfrentar',
 'discursa',
 'procura',
 'relator',
 'pregado',
 'conduzindo',
 'reguladoras',
 'acumulou',
 'adiantará',
 'podiam',
 'instalar',
 'embora',
 'decreto',
 'presidentes',
 'chame',
 'declarações',
 'preservação',
 'deixei',
 

In [6]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(strip_accents= "unicode", vocabulary= lexicon, binary = False))
                    ])

print("USING FEATURE FREQUENCY")
evaluate(all_data, features, 10)

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(strip_accents= "unicode", vocabulary= lexicon, binary = True))
                    ])
print("USING FEATURE PRESENCE")
evaluate(all_data, features, 10)

USING FEATURE FREQUENCY
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.803118712274
Desvio padrão:  0.0388411548861

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.781770623742
Desvio padrão:  0.0370706750263

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.754688128773
Desvio padrão:  0.0350200961587
USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.805955734406
Desvio padrão:  0.0421287032539

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.778913480885
Desvio padrão:  0.0455876117809

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.746076458753
Desvio padrão:  0.0365771304748


In [8]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words= stopwords_pt)

neg_counts = cv.fit_transform(neg_train["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_train["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())
len(lexicon_bigrams)

14252

In [9]:
from pandas import Series
s = Series(neg_excl_bigrams)
s.nlargest(15)

candidata marina    10
deputados           10
minas energia       10
acordo delacao       8
agua                 7
criticar             7
criticou             7
henrique             7
presidentes          7
senadores            7
alves                6
calheiros            6
camara               6
dilma diz            6
edison               6
dtype: int64

In [10]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

print("USING FEATURE FREQUENCY")
evaluate(all_data, features, 10)

print("\nUSING FEATURE PRESENCE")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams, binary= True))
                    ])
evaluate(all_data, features, 10)

USING FEATURE FREQUENCY
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.7846277666
Desvio padrão:  0.0361520986916

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.858752515091
Desvio padrão:  0.0303525645053

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.764668008048
Desvio padrão:  0.0341743498897

USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.813158953722
Desvio padrão:  0.0356105217482

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.833098591549
Desvio padrão:  0.0388677245923

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.718973843058
Desvio padrão:  0.0338068946951


In [11]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", TfidfVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

print("USING FEATURE FREQUENCY")
evaluate(all_data, features, 10)

print("\nUSING FEATURE PRESENCE")
features = FeatureUnion([
                    ("lexicon_vector", TfidfVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams, binary= True))
                    ])
evaluate(all_data, features, 10)

USING FEATURE FREQUENCY
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.776056338028
Desvio padrão:  0.0342645951903

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.825975855131
Desvio padrão:  0.0436444855526

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.557766599598
Desvio padrão:  0.00187122736419

USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.776036217304
Desvio padrão:  0.0383353872726

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.824527162978
Desvio padrão:  0.0438520512715

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.557766599598
Desvio padrão:  0.00187122736419


In [12]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest,chi2, mutual_info_classif
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = Pipeline([
                    ("lexicon_vector", CountVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams)),
                    ("feature_selection", TruncatedSVD(n_components = 6000))
                    ])

run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds = 10, shuffle= True)
print("\nUSING FEATURE PRESENCE")
features = Pipeline([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams, binary= True)),
                    ("feature_selection", VarianceThreshold(threshold = 0.01))
                    ])
evaluate(all_data, features, 10)

Cross Validation:
Accuracia media:  0.863038229376
Desvio padrão:  0.0314826074733

USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.573460764588
Desvio padrão:  0.0149667491065

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.589134808853
Desvio padrão:  0.0130704200573

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.574869215292
Desvio padrão:  0.0193800180634


In [3]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest,chi2, mutual_info_classif
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = Pipeline([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams)),
                    ("feature_selection", TruncatedSVD(n_components = 6000))
                    ])

run_cross_validation2(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds = 10, shuffle= True)

Cross Validation:


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [4]:
all_data

,labels,texts
5,NG,o presidente do pt rui falcão acusou na noit...
6,NG,aécio faz coro a denúncias sem provas veicula...
7,NG,segundo falcão é cômico ouvir alguém do ps...
8,NG,ele citou o mensalão mineiro o esquema de cor...
9,NG,para o presidente do pt aécio deveria se preo...
10,NG,a gravidade das acusações do candidato tucano...
12,NG,o brasil acordou hoje perplexo com as mais gr...
13,NG,na campanha de dilma rousseff a primeira reaç...
14,NG,a preocupação é justificável quando a polícia...
16,NG,os dois nomes mais citados são o tesoureiro pe...


In [16]:
type(M)

numpy.ndarray